In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
sample_size = 1000000
max_num = 1000

In [3]:
fib_data = pd.DataFrame(
    {'x1': np.random.rand(sample_size), 
     'x2': np.random.rand(sample_size)})
fib_data['y'] = fib_data['x1'] + fib_data['x2']

In [4]:
fib_data.head()

,x1,x2,y
0,0.698746,0.704676,1.403423
1,0.675440,0.758063,1.433503
2,0.881600,0.615355,1.496955
3,0.401164,0.540395,0.941559
4,0.448959,0.082627,0.531586


## Let's train it & vizualize  :)

In [5]:
## Network configuration
learning_rates = [1, 0.1, 0.01, 0.001]

with tf.name_scope("input_data"):
    input_numbers = tf.placeholder(tf.float32, shape=[None, 2], name="input")
    output_numbers = tf.placeholder(tf.float32, shape=[None, 1], name="output")
    learning_rate = tf.placeholder(tf.float32)

with tf.name_scope("layer"):
    weight = tf.Variable(tf.truncated_normal([2, 1]), dtype=tf.float32, name="weight")
    bias = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="bias")
    prediction = tf.matmul(input_numbers, weight) + bias 
    tf.summary.histogram('weight', weight)
    tf.summary.histogram('bias', bias)
    tf.summary.histogram('prediction', prediction)
    
    
with tf.name_scope("evaluation"):
    loss = tf.reduce_mean(tf.square(prediction - output_numbers), name="loss")
    tf.summary.scalar('loss', loss)
    

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)

In [6]:
epochs = 500
feed_dict = {input_numbers: fib_data[['x1', 'x2']], output_numbers: fib_data[['y']]}
init = tf.global_variables_initializer()

In [7]:
with tf.Session() as sess:
    for lr in learning_rates:
        print('learning rate', lr)
        feed_dict[learning_rate] = lr
        sess.run(init)    
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("logs/02/"+str(lr))
        writer.add_graph(sess.graph)
    
        for epoch in range(epochs):
            if epoch % 50 == 0:
                print('loss', sess.run(loss, feed_dict))
            if epoch % 10 ==0:
                loss_so_far = sess.run(loss, feed_dict)
                if np.isnan(loss_so_far):
                    break
                s = sess.run(merged_summary, feed_dict)
                writer.add_summary(s, epoch)
            sess.run(train, feed_dict)
        print(sess.run([weight, bias], feed_dict))
        a = sess.run(prediction, feed_dict)

learning rate 1
loss 2.19915
loss 3.50853e+31
loss inf
[array([[ nan],
       [ nan]], dtype=float32), array([ nan], dtype=float32)]
learning rate 0.1
loss 0.293302
loss 0.0145884
loss 0.00343475
loss 0.000879266
loss 0.000243745
loss 7.21072e-05
loss 2.23504e-05
loss 7.14229e-06
loss 2.32542e-06
loss 7.65518e-07
[array([[ 0.998914  ],
       [ 0.99865645]], dtype=float32), array([ 0.00128457], dtype=float32)]
learning rate 0.01
loss 1.61512
loss 0.204162
loss 0.124658
loss 0.106925
loss 0.093767
loss 0.082383
loss 0.0724401
loss 0.063744
loss 0.0561319
loss 0.0494639
[array([[ 0.84487349],
       [ 0.29834235]], dtype=float32), array([ 0.45316821], dtype=float32)]
learning rate 0.001
loss 0.386006
loss 0.333427
loss 0.293917
loss 0.264051
loss 0.241286
loss 0.223769
loss 0.210125
loss 0.199348
loss 0.190689
loss 0.183607
[array([[ 1.69247437],
       [-0.25732681]], dtype=float32), array([ 0.20478284], dtype=float32)]
